<a href="https://pytorch.org/"><img src="https://raw.githubusercontent.com/pytorch/pytorch/master/docs/source/_static/img/pytorch-logo-dark.png" width="600px"/></a>

# **PyTorch** (1/2): the world's simplest neural net from scratch

In this notebook, we will build a simple neural network to learn a linear equation with one variable a.k.a. a _line_.

## What is PyTorch?
* open-source machine learning library
* developed by Facebook AI Research + community
* used in most state-of-the-art research
* "Python first": deeply integrated into Python, it should "feel familiar" to use

### Main Components
* ```torch```: tensor library (like NumPy but with GPU support)
* ```torch.autograd```: automatic differentiation library
* ```torch.jit```: compile PyTorch code to TorchScript for deployment (e.g. in standalone C++ program)
* ```torch.nn```: neural network library

## Useful Links
* [🔗 Official PyTorch Documentation](https://pytorch.org/docs/stable/index.html)

Tutorials:
* [🔗 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html)
* [🔗 PyTorch Tutorials with Examples](https://pytorch.org/tutorials/beginner/pytorch_with_examples.html)

## Attribution

Most of the material in the theoretical part of this document is taken from the fast.ai [Practical Deep Learning for Coders](https://course.fast.ai/) notebooks.

In you are interested, check out the colab notebooks from the fast.ai course, especially
* [01_intro](https://colab.research.google.com/github/fastai/fastbook/blob/master/01_intro.ipynb) and 
* [04_mnist_basics](https://colab.research.google.com/github/fastai/fastbook/blob/master/04_mnist_basics.ipynb). 

They cover a lot of fundamentals of neural networks and machine learning in general in more depth.

# Training a Neural Network to Learn a Line

## Creating our Dataset

In [ ]:
%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import torch

As we have seen in the ML introduction, we need some data to train our model.

In our case, we can just generate data points for an arbitrary line function. In order to do this we first need to define our _target_, i.e. the line equation that we want our model to learn. 

As you might recall, a _line_ is defined by the equation $$ y = ax + b $$

It has one variable ($x$) and two parameters:
* $a$ (_slope_) and 
* $b$ (_intercept_). 

The _parameters_ are what we want our model to learn.

Let's create an arbitrary line function by generating some random values for $a$ and $b$.

When developing algorithms that contain any kind of randomness, for reproducibility it is always a good idea to set a _manual seed_ before you start. This will initialize the random number generators to the same state every time you run your experiment, i.e. you will always get the same results.

In NumPy, you can do this by passing an arbitrary number to the function `numpy.random.seed()`.

In [ ]:
seed = 42
np.random.seed(seed) # for reproducibility

In [ ]:
a, b = np.random.uniform(low=-2.0, high=2.0, size=2)
a, b

We use these to define our target function $f$:

In [ ]:
def f(x):
    return a*x + b

Next, we create our _independent variable_ $x$ by generating 1000 evenly spaced numbers over the interval $[-\pi, \pi]$.

In [ ]:
n_samples = 5000

x = np.linspace(-np.pi, np.pi, n_samples, dtype=np.float32)

Note that the number of samples we generate will influence our results: the more data points we allow the model to learn from, the better our final performance will be.

Let's create our _dependent variable_ $y$ by passing $x$ to our function $f$:

In [ ]:
y_true = f(x)

In the real world, any data that you obtain will include some kind of noise (e.g. slight deviations in temperature readings due to a given precision of the sensor itself). Luckily, neural networks can handle noisy data quite well. Actually, introducing noise during training (especially for small datasets) can improve the robustness of the network and result in a better generalization (see e.g. [here](https://machinelearningmastery.com/train-neural-networks-with-noise-to-reduce-overfitting/)).

So, let's go ahead and add some noise to our _true_ $y$ data.

Now, we can draw some samples from a normal distribution and add it to our true $y$.

In [ ]:
e = np.random.normal(size=y_true.shape).astype(np.float32)
y_noisy = y_true + e

Let's have a look what our line looks like.

In [ ]:
fig, ax = plt.subplots()

ax.scatter(x, y_noisy, s=1, color='#90a4ae', alpha=0.75, label='y_noisy')
ax.plot(x, y_true, color='#455a64', label='y_true')
ax.grid(True, alpha=0.4)
ax.legend()
fig.tight_layout();

## Defining our Model

If we want a neural network to learn this function, we need a model that also has (at least) two parameters. 

But first we need to decide what _kind_ of layers our neural network is made of. Since we want to model a linear equation, `torch.nn.Linear` might be a good fit. Let's have a look at [what this layer does](https://pytorch.org/docs/master/generated/torch.nn.Linear.html#torch.nn.Linear).

This is exactly what we need! Conceptually, it looks like this:

<img src="../assets/images/linear_unit.svg" alt="Linear unit" width="400px"/>



We have $n$ inputs $x_1, x_2, \dots, x_n$, each of which is multiplied by its corresponding weight $w_1, w_2, \dots, w_n$. The output $y$ is the weighted sum of all inputs plus a single bias term $b$.

For our specific case, we can simplify this to:

<img src="../assets/images/linear_unit_single.svg" alt="Linear unit with a single input" width="400px"/>

We just have a single input ($x$), the weight corresponds to the slope $a$ and the bias $b$ corresponds to the intercept.

To construct an instance of `torch.nn.Linear`, we need specify the sizes of our input and output samples. In our case, these are just scalar values (i.e. shape `1`).

Again, we set a manual seed here because the layer object takes care of randomly initializing its parameters. The PyTorch way of doing this is by calling `torch.manual_seed()`.

In [ ]:
torch.manual_seed(seed); # for reproducibility

model = torch.nn.Sequential(
    torch.nn.Linear(1, 1),
    torch.nn.Flatten(0, 1)
)

We also add a `Flatten` layer at the end. This will flatten the output to a 1D tensor to match the shape of our labels (i.e. the `y_noisy` vector).

We can have have a look at our model's parameters:

In [ ]:
[(name, tensor.item()) for (name, tensor) in model.named_parameters()]

As expected, we have two scalar parameters: a _weight_ and a _bias_.

Currently, our training data is stored in NumPy arrays. To be able pass the data to our model, we have to convert them to PyTorch tensors:

In [ ]:
xx = torch.from_numpy(x).unsqueeze(-1)
yy = torch.from_numpy(y_noisy)

xx.shape, yy.shape

## Constructing the Training Loop

As mentioned before, our model parameters are initialized to random values. 

Let's see how well our untrained neural network is predicting our line.

In [ ]:
n_training_steps = 100

In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(6, 6), gridspec_kw={'height_ratios':[0.7, 0.3]})
axes[0].scatter(x, y_noisy, s=0.5, color='#90a4ae', alpha=0.25, label='y_noisy')
axes[0].plot(x, y_true, linewidth=1, label='y_true', color='#455a64')
axes[0].set_title('Step 0')
line_pred = axes[0].plot(x, model(xx).detach().numpy(), label='y_pred', color='#ffa000')[0]
line_loss = axes[1].semilogy(np.arange(n_training_steps)[0], 1000, label='training loss')[0]
axes[1].set_xlim([0, n_training_steps])
axes[1].set_xlabel('# epoch')
axes[1].set_ylabel('mse loss')
for ax in axes:
    ax.grid(True, which='both', alpha=0.25)
    ax.legend()
fig.canvas.header_visible = False
fig.tight_layout()

def update_every(iteration, every = 50):
    return iteration % every == every - 1

def set_plot_data(y_pred, loss):
    line_pred.set_ydata(y_pred)
    line_loss.set_data(np.arange(loss.shape[0]), loss)
    axes[0].set_title(f'Epoch {t}: loss {loss[-1]:.2f}')
    axes[1].set_xlim([0, n_training_steps])
    axes[1].relim()
    axes[1].autoscale_view()
    fig.canvas.draw()

Let's remind ourselves of the <a href="#detailed_loop">Detailed Training Loop Diagram</a> and compare what we have achieved so far:

* [x] Collect input data (our tensor `xx`) and
* [x] Corresponding label data (our tensor `yy`)
* [x] Define a model architecture
* [x] Initialize model parameters to random values
* [x] Calculate predictions (our tensor `y_pred`) based on current model parameters and input data
* [ ] Define a loss function to determine the performance
* [ ] Update model parameters based on loss using automated process (Stochastic Gradient Descent)


To update our model parameters, we need to define a suitable loss function. 

We want `y_pred` to exactly match `y_true`, so we need a mathematical way of figuring out how _similar_ our prediction is to our target line.

We can use the _mean squared error (MSE)_ to achieve this. It is defined like this:

$\text{MSE} = \frac{1}{n} \sum_{i = 1}^{n} (Y_i - \hat{Y_i})^2$

For every data sample $i$, we compute the squared difference of the target value $Y_i$ and the predicted value $\hat{Y_i}$. The squared differences are then added up accross all sample and divided by the number of samples to yield a single scalar value, the MSE.

In [ ]:
loss_fn = torch.nn.MSELoss(reduction='sum')

#### Stochastic Gradient Descent (SGD)

Furthermore, we need the automated process that will find values for our model parameters $w$ and $b$, such that the predictions closely resemble our target line. This is what the SGD algorithm does for us.

Specifically, these are the steps that are required to make our model learn from its experience:

<br>
<img src="../assets/images/sgd.svg" alt="Stochastic Gradient Descent process" width="700px"/>
<a href="https://colab.research.google.com/github/fastai/fastbook/blob/master/04_mnist_basics.ipynb">(Image Source)</a>
<br>

1. *Initialize* the weights.
1. For each input sample, use these weights to *predict* the output value.
1. Based on these predictions, calculate how good the model is (its *loss*).
1. Calculate the *gradient*, which measures for each weight, how changing that weight would change the loss
1. *Step* (that is, change) all the weights based on that calculation.
1. Go back to the step 2, and *repeat* the process.
1. Iterate until you decide to *stop* the training process (for instance, because the model is good enough or you don't want to wait any longer).

Essentially, we want SGD to find the (_a_) minimum of our loss function.

<img src="https://mlfromscratch.com/content/images/2019/12/gradient-descent-optimized--1-.gif" alt="SGD" width="600px"/>

Example from [here](https://mlfromscratch.com/optimizers-explained/#/) (animation originally from [3blue1brown](https://www.youtube.com/watch?v=IHZwWFHWa-w)).

In [ ]:
learning_rate = 1e-5

n_training_steps = 100
losses = np.ones(n_training_steps) * np.infty

torch.manual_seed(seed)
model[0].reset_parameters()

t = 0
set_plot_data(model(xx).detach().numpy(), losses[:1])

In [ ]:
t += 1

# run forward pass
y_pred = model(xx)

# calculate loss
loss = loss_fn(y_pred, yy)

# update plots
losses[t] = loss.item()
if update_every(t, 1):
    set_plot_data(y_pred.detach().numpy(), losses[:t+1])

# zero out any previous gradients
model.zero_grad()

# compute gradient of loss w.r.t model parameters
loss.backward()

# update the parameters using SGD
with torch.no_grad():
    for param in model.parameters():
        param -= learning_rate * param.grad

Let's actually go ahead and turn this into a loop

In [ ]:
test = next(model.parameters())

In [ ]:
test

In [ ]:
test.item()

In [ ]:
learning_rate = 1e-3

n_training_steps = 1000
losses = np.ones(n_training_steps) * np.infty

torch.manual_seed(seed)
model[0].reset_parameters()
set_plot_data(model(xx).detach().numpy(), losses[:1])


for t in range(n_training_steps):       
    # run forward pass
    y_pred = model(xx)
    
    # calculate loss
    loss = loss_fn(y_pred, yy)

    # update plots if needed
    losses[t] = loss.item()
    if update_every(t, 10):
        set_plot_data(y_pred.detach().numpy(), losses[:t+1])

    # zero out any previous gradients
    model.zero_grad() 
    
    # compute gradient of loss w.r.t model parameters
    loss.backward() 

    # update parameters
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad
            
set_plot_data(y_pred.detach().numpy(), losses)

We can use `IPython.display` to display a png copy of our figure:

In [ ]:
display(fig)

#### Using Optimizers

We can use optimizers to perform the parameter update step automatically.

In [ ]:
learning_rate = 1e-5
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
learning_rate = 0.03
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
n_training_steps = 200

loss_fn = torch.nn.MSELoss()
losses = np.ones(n_training_steps) * np.infty

#parameters = np.zeros((n_training_steps, 2))

torch.manual_seed(seed)
model[0].reset_parameters()
set_plot_data(model(xx).detach().numpy(), losses[:1])

In [ ]:
for t in range(n_training_steps):
    #parameters[t] = [p.item() for p in model.parameters()]
    
    # run forward pass
    y_pred = model(xx)
    
    # calculate loss
    loss = loss_fn(y_pred, yy)

    # update plots if needed
    losses[t] = loss.item()
    if update_every(t, 5):
        set_plot_data(y_pred.detach().numpy(), losses[:t+1])

    # zero out any previous gradients
    optimizer.zero_grad() 
    
    # compute gradient of loss w.r.t model parameters
    loss.backward() 

    # update parameters
    optimizer.step()
    
set_plot_data(y_pred.detach().numpy(), losses)

If we have a look at the model parameters again, we can verfiy that they now closely resemble our _target_ parameters:

In [ ]:
[param[1].item() for param in model.named_parameters()]

In [ ]:
[a, b] # target parameters

For visualization of the behaviour of different optimizers, check out [this](https://distill.pub/2017/momentum/), [this](https://bl.ocks.org/EmilienDupont/aaf429be5705b219aaaf8d691e27ca87) and [this](https://ikocabiyik.com/blog/en/visualizing-ml-optimizers/).

# Plotting the Weight Landscape

In [ ]:
torch.manual_seed(seed); # for reproducibility

model = torch.nn.Sequential(
    torch.nn.Linear(1, 1),
    torch.nn.Flatten(0, 1)
)

In [ ]:
n_mesh = 200

xv, yv = np.meshgrid(
    np.linspace(a - 2, a + 2, n_mesh), 
    np.linspace(b - 2, b + 2, n_mesh))

In [ ]:
xv.shape, yv.shape

In [ ]:
xxv = torch.from_numpy(xv)
yyv = torch.from_numpy(yv)

In [ ]:
zzz = torch.zeros_like(xxv)

In [ ]:
with torch.no_grad():
    for i, (a_, b_) in enumerate(zip(xxv.flatten(), yyv.flatten())):
        pa, pb = model.parameters()
        pa.copy_(torch.tensor([[a_]]))
        pb.copy_(torch.tensor([b_]))
        loss = loss_fn(model(xx), yy)
        zzz.flatten()[i] = loss

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
from matplotlib import cm
from matplotlib.ticker import LinearLocator, LogLocator, FormatStrFormatter

fig = plt.figure()
ax = fig.gca(projection='3d')

# Plot the surface.
min_loss = zzz.min().item()
max_loss = zzz.max().item()
surf = ax.plot_surface(xxv.numpy(), yyv.numpy(), zzz.numpy(),
                       cmap=cm.inferno,
                       norm=mpl.colors.LogNorm(vmin=min_loss, vmax=max_loss),
                       antialiased=False)

idx_min = zzz.argmin()
ax.scatter3D(xxv.flatten()[idx_min], yyv.flatten()[idx_min], zzz.flatten()[idx_min], c='r',  zorder=10)

ax.set_xlabel('a')
ax.set_ylabel('b')
fig.tight_layout()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import ma
from matplotlib import ticker, cm

# Automatic selection of levels works; setting the
# log locator tells contourf to use a log scale:
fig, ax = plt.subplots()
cs = ax.contourf(xxv.numpy(), yyv.numpy(), zzz.numpy(), 
                
                levels=np.logspace(np.log10(min_loss), np.log10(max_loss), 50),
                 #levels=np.linspace(min_loss, max_loss, 50),
                norm=mpl.colors.LogNorm(vmin=min_loss, vmax=max_loss),
                 #locator=ticker.LogLocator(),
                 cmap=cm.inferno,
                )

idx_min = zzz.argmin()
ax.scatter(xxv.flatten()[idx_min], yyv.flatten()[idx_min], c='r')

ax.set_xlabel('a')
ax.set_ylabel('b')
#cbar = fig.colorbar(cs)

In [ ]:
ax.plot(parameters[:, 0], parameters[:, 1], linewidth=1)

In [ ]:
parameters.shape

# A Closer Look: Automatic differentiation with ```torch.autograd```

Adapted from [here](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#sphx-glr-beginner-blitz-autograd-tutorial-py).

We create two tensors `a` and `b` with `requires_grad=True`. This signals to `autograd` that every operation on them should be tracked.

In [ ]:
a = torch.tensor([4.], requires_grad=True)
b = torch.tensor([2.], requires_grad=True)

We create another tensor ```y``` from ```a``` and ```b```:

$ y = 3a^3 - b^2$

In [ ]:
y = 3 * a**3 - b**2
y

Let's assume ```a``` and ```b``` to be parameters of a neural network, and ```y``` to be the error. In NN training, we want gradients of the error w.r.t the parameters, i.e.

$ \frac{\partial y}{\partial a} = 9a^2 $ and $\frac{\partial y}{\partial b} = -2b $

When we call ```.backward()``` on ```y```, autograd calculates these gradients and stores them in the respective tensors' ```.grad``` attribute.

In [ ]:
# grads = torch.tensor([1., 1., 1.])
# y.backward(gradient=grads)
y.backward()

We need to explicitly pass a ```gradient``` argument in ```y.backward()``` if ```y``` is a vector. ```gradient``` is a tensor of the same shape as ```y```, and it represents the gradient of ```y``` w.r.t itself, i.e.

$ \frac{\partial y}{\partial y} = 1 $

Note: the ```gradient``` argument only has to be specified if ```y``` is a non-scalar (i.e. its data has more than one element), otherwise it can be created implicitly.

Think of the ```gradient``` argument as a boolean vector specifying for which elements of ```y``` we want the gradients to be calculated.

The gradients are now deposited in ```a.grad``` and ```b.grad```.

In [ ]:
a.grad, b.grad

Let's check that they are actually correct:

In [ ]:
a.grad == 9 * a**2

In [ ]:
b.grad == -2 * b